# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
ed.links

['https://edwarddonner.com/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/10/16/from-soft

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [11]:
link_system_prompt = (
    "You are provided with a list of links found on a webpage. "
    "You will decide which links are most relevant to include in a brochure about the company, "
    "such as links to an About page, Careers/Jobs pages, or other key pages. "
    "Do not include links to Terms of Service, Privacy Policy, or email addresses. "
    "Respond in JSON format as shown in these examples:\n\n"
)

link_system_prompt += """
Example 1:
Input:
Here is the list of links on the website of https://edwarddonner.com. Decide which links are relevant for a brochure about the company:
Links:
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/

Output:
{
    "links": [
        {"type": "about page", "url": "https://edwarddonner.com/about-me-and-about-nebula/"},
        {"type": "home page", "url": "https://edwarddonner.com/"}
    ]
}

Example 2:
Input:
Here is the list of links on the website of https://huggingface.co. Decide which links are relevant for a brochure about the company:
Links:
'/',
'/models',
'/datasets',
'/docs',
'/docs/transformers',
'/docs/diffusers',
'/pricing',
'/enterprise',
'/terms-of-service',
'/huggingface',
'https://apply.workable.com/huggingface/',
'/privacy',
'/blog',
'mailto:press@huggingface.co',
'https://status.huggingface.co/',
'https://github.com/huggingface',

Output:
{
    "links": [
        {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
        {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
        {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
        {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
        {'type': 'blog page', 'url': 'https://huggingface.co/blog'}
    ]
}
"""

In [12]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You will decide which links are most relevant to include in a brochure about the company, such as links to an About page, Careers/Jobs pages, or other key pages. Do not include links to Terms of Service, Privacy Policy, or email addresses. Respond in JSON format as shown in these examples:


Example 1:
Input:
Here is the list of links on the website of https://edwarddonner.com. Decide which links are relevant for a brochure about the company:
Links:
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/

Output:
{
    "links": [
        {"type": "about page", "url": "https://edwarddonner.com/about-me-and-about-nebula/"},
        {"type": "home

In [13]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [14]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/
https:/

In [15]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [16]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

['/',
 '/models',
 '/datasets',
 '/spaces',
 '/posts',
 '/docs',
 '/enterprise',
 '/pricing',
 '/login',
 '/join',
 '/deepseek-ai/DeepSeek-R1',
 '/hexgrad/Kokoro-82M',
 '/deepseek-ai/DeepSeek-R1-Distill-Qwen-32B',
 '/deepseek-ai/DeepSeek-R1-Zero',
 '/deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B',
 '/models',
 '/spaces/hexgrad/Kokoro-TTS',
 '/spaces/tencent/Hunyuan3D-2',
 '/spaces/lllyasviel/iclight-v2',
 '/spaces/webml-community/deepseek-r1-webgpu',
 '/spaces/JeffreyXiang/TRELLIS',
 '/spaces',
 '/datasets/fka/awesome-chatgpt-prompts',
 '/datasets/cais/hle',
 '/datasets/bespokelabs/Bespoke-Stratos-17k',
 '/datasets/HumanLLMs/Human-Like-DPO-Dataset',
 '/datasets/yale-nlp/MMVU',
 '/datasets',
 '/join',
 '/pricing#endpoints',
 '/pricing#spaces',
 '/pricing',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/enterprise',
 '/allenai',
 '/facebook',
 '/amazon',
 '/google',
 '/Intel',
 '/microsoft',
 '/grammarly',
 '/Writer',
 '/docs/transformers',

In [17]:
get_links("https://edwarddonner.com")

{'links': [{'type': 'home page', 'url': 'https://edwarddonner.com/'},
  {'type': 'about page',
   'url': 'https://edwarddonner.com/about-me-and-about-nebula/'},
  {'type': 'blog page', 'url': 'https://edwarddonner.com/posts/'},
  {'type': 'twitter page', 'url': 'https://twitter.com/edwarddonner'},
  {'type': 'facebook page',
   'url': 'https://www.facebook.com/edward.donner.52'}]}

In [18]:
get_links("https://huggingface.co")

{'links': [{'type': 'home page', 'url': 'https://huggingface.co/'},
  {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'},
  {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'},
  {'type': 'blog page', 'url': 'https://huggingface.co/blog'},
  {'type': 'about page', 'url': 'https://huggingface.co/huggingface'},
  {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'},
  {'type': 'community page', 'url': 'https://discuss.huggingface.co'},
  {'type': 'GitHub page', 'url': 'https://github.com/huggingface'},
  {'type': 'LinkedIn page',
   'url': 'https://www.linkedin.com/company/huggingface/'}]}

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [19]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [ ]:
print(get_all_details("https://huggingface.co"))

In [29]:
# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."


In [22]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:5_000] # Truncate if more than 5,000 characters
    return user_prompt

In [23]:
get_brochure_user_prompt("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'discussion forum', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}]}


'You are looking at a company called: HuggingFace\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHugging Face – The AI community building the future.\nWebpage Contents:\nHugging Face\nModels\nDatasets\nSpaces\nPosts\nDocs\nEnterprise\nPricing\nLog In\nSign Up\nThe AI community building the future.\nThe platform where the machine learning community collaborates on models, datasets, and applications.\nTrending on\nthis week\nModels\ndeepseek-ai/DeepSeek-R1\nUpdated\n2 days ago\n•\n109k\n•\n2.49k\nhexgrad/Kokoro-82M\nUpdated\nabout 21 hours ago\n•\n35.2k\n•\n2.38k\ndeepseek-ai/DeepSeek-R1-Distill-Qwen-32B\nUpdated\n2 days ago\n•\n86.1k\n•\n465\ndeepseek-ai/DeepSeek-R1-Zero\nUpdated\n2 days ago\n•\n5.42k\n•\n424\ndeepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B\nUpdated\n2 days ago\n•\n94k\n•\n366\nBrowse 400k+ models\nSpaces\nRunning\non\nZero\n1.52k\n❤️\nKokoro TTS\nN

In [24]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [25]:
create_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.com/'}, {'type': 'about page', 'url': 'https://huggingface.com/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.com/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.com/pricing'}, {'type': 'blog page', 'url': 'https://huggingface.com/blog'}, {'type': 'forum page', 'url': 'https://discuss.huggingface.co'}, {'type': 'status page', 'url': 'https://status.huggingface.co/'}, {'type': 'github page', 'url': 'https://github.com/huggingface'}, {'type': 'twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'linkedin page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# Hugging Face Company Brochure

## About Us
Welcome to **Hugging Face**, the AI community building the future! We are a leading platform for collaboration in the machine learning community, where users can come together to share and develop **models**, **datasets**, and **applications**. With over 400,000 models and 100,000 datasets available, we provide the tools needed to advance machine learning technology.

## Our Offerings
- **Models**: Explore and contribute to a diverse range of advanced AI models.
- **Datasets**: Access comprehensive datasets for training and evaluation of machine learning models.
- **Spaces**: Create, share, and showcase your applications with a vibrant community.

## Enterprise Solutions
We offer **Enterprise solutions** for businesses aimed at enhancing their AI initiatives with:
- Dedicated support and enterprise-grade security
- Access to optimized compute infrastructure
- Custom features to meet your team's needs

**Starting at just $20/user/month**, organizations can leverage our powerful platform for their AI projects.

## Customers
More than **50,000 organizations** trust Hugging Face, including industry leaders such as:
- **Meta**
- **Amazon Web Services**
- **Google**
- **Microsoft**
- **Intel**
- **Grammarly**

## Company Culture
At Hugging Face, we pride ourselves on a **collaborative and inclusive culture**. Our team is passionate about open-source technology and believes in the power of community to drive innovation. We encourage creativity, diverse ideas, and continuous learning as we strive to push the boundaries of what AI can do.

## Careers at Hugging Face
We are always on the lookout for talented individuals to join our team. If you're passionate about AI and want to be part of a community that shapes the future, check out our careers page for job openings that match your skills and aspirations.

- **Learn More**: Join our community on [GitHub](https://github.com) or follow us on [Twitter](https://twitter.com) and [LinkedIn](https://linkedin.com) for the latest updates and openings.

### Join Us!
Are you ready to accelerate your machine learning journey? Sign up today on our platform to start collaborating, sharing, and building the future of AI! 

For more information, visit our [website](https://huggingface.co).

---

**Hugging Face** - The AI community building the future.

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [26]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [28]:
stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}]}


# Hugging Face Brochure

---

## About Us
**Hugging Face** is a pioneering AI community dedicated to building the future of machine learning through collaboration. Our platform allows developers, researchers, and enthusiasts to share and work on models, datasets, and applications, creating an ecosystem where innovation thrives. 

## Our Offerings
- **Models**: Access over **400,000 models**, including popular choices like **DeepSeek-R1** and applications across various domains.
- **Datasets**: Browse **100,000+ datasets** suitable for different machine learning needs, from natural language processing to computer vision.
- **Spaces**: Create and host applications seamlessly. Our **Spaces** feature allows for interactive demos and applications using machine learning models.

## Community
We pride ourselves on being "The Home of Machine Learning." Hugging Face empowers more than **50,000 organizations**, including industry leaders like **Google**, **Microsoft**, and **Amazon Web Services**. Our community includes individuals and teams from non-profits and enterprises alike, collaborating to accelerate their projects.

## Company Culture
We believe in open-source, collaboration, and transparency. Our community-driven culture fosters an environment where diverse talents can come together to enhance machine learning tools and technologies. From casual interactions in our forums to collaborative projects, we encourage creativity and innovation.

## Careers
Join our growing team passionate about AI and machine learning! We offer opportunities across various roles, from engineering and product management to customer success and marketing. At Hugging Face, we prioritize creating a supportive and inclusive workplace where every voice is heard.

For a detailed job listing and to learn more, visit our [Careers page](#).

## Pricing
We offer accessible solutions with the following starting prices:
- **Compute**: Starting at $0.60/hour for GPU.
- **Enterprise Solutions**: Starting at $20/user/month, equipped with enterprise-grade security and dedicated support.

---

**Join Us**  
Contribute to the future of AI with Hugging Face. Whether you’re a developer looking to showcase your models, a business seeking enterprise solutions, or a recruit aiming to be part of an innovative team, we welcome you!

--- 

For more information, visit our website at [Hugging Face](https://huggingface.co) or connect with us on [Twitter](#), [LinkedIn](#), and [Discord](#).  

Together, let's build the future of AI!

In [30]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("HuggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'home page', 'url': 'https://huggingface.co/'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'pricing page', 'url': 'https://huggingface.co/pricing'}, {'type': 'documentation page', 'url': 'https://huggingface.co/docs'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'community discussion', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'learn page', 'url': 'https://huggingface.co/learn'}]}



# Welcome to Hugging Face: Where AI Hugs You Back!

## 🤖 Join the AI Revolution!
Oh, hey there! Are you ready to join the most *hug-tastic* community on the planet? That's right—Hugging Face is where the AI wizards, data nerds, and machine learning aficionados gather to create, collaborate, and occasionally snack on data cookies. We’re building the future, one model at a time, and we promise it’s a whole lot better than your last Zoom call!

---

## 🎉 The Hugging Face Culture

### **Our Motto: Share a Model, Share a Hug!**
At Hugging Face, we live by the rule that sharing is caring! Whether you’re a lone wolf coder or part of a massive enterprise (just like Amazon, Google, and *that* friend who won't stop talking about their “latest data obsession”), everyone’s welcome. 

### **Feeling Chill?**
- **Friday Game Nights**: Nothing says “AI breakthrough” like a rousing game of Among Us or a round of virtual ping pong. 
- **Snack Club**: Yes, we have data cookies. And yes, you have to bring your own cookies to join.

---

## 🌍 Who do we serve? 

With over **50,000 organizations** on board, including some really big kids like Microsoft, Grammarly, and even your uncle's tech startup, we’re the playground where machine learning meets imagination. Our customers range from individual AI pioneers to multinational tech giants. Basically, if you like nerdy stuff and believe in the potential of AI, then you're our kind of people!

---

## 💼 Career Opportunities: The More the Merrier!

### **Spotlight: The Hugging Face Team!** 
Want to be part of a community that’s as diverse as the datasets we manage? We’re looking for talented folks to join our family! Here’s what you might do:
- **Model Wrangler**: Do you have what it takes to train the next breakthrough AI model? We need you to help tame the wild world of algorithms!
- **Dataset Diva/Dude**: Filter through endless data while wearing your favorite data-processing superhero cape. 
- **Collaboration Wizard**: Help companies embrace their inner hugger—because let’s face it, every company needs a great hug!

## 🤗 Let’s Get Started!

### **Hugging Face Open Beta:**
- Sign up for FREE and unlock the door to a treasure trove of **400k+ models**, **100k+ datasets**, and something even more elusive—team spirit!
- Your journey on this platform can lead to **GPU-powered dreams** starting at just **$0.60/hour**!

---

## ✨ The Future Awaits

So, whether you’re looking to sharpen your AI skills, create epic models, or just trying to figure out what a “transformer” really is (not the robot kind), we’ve got you covered! Join us, and remember: We don’t just build AI; we give it the best hugs in the industry!

### **Come on, let’s Hug it Out!**



<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>